In [1]:
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import os
import sys
import time
import torch
from torch.autograd import Variable
import torch.nn as nn

import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse

## Argument Declare

In [2]:
DATASET_ROOT = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN"
DATASET_NAME = "unity"
cfg = mmfcn
BASE_NET = "./weights/vgg16_reducedfc.pth"
DATA_DETECTION = MMFCNDetection
BATCH_SIZE = 32
#PRETRAINED_MODEL = "/home/arg_ws3/ssd.pytorch/weights/person/person_67500.pth"
PRETRAINED_MODEL = None
PRETRAINED_EPOCH = 0
SAVE_MODEL_EPOCH = 1
START_ITER = 0
NUM_WORKERS = 8
EPOCH = 61
adjust_lr_epoch = [15, 30, 45]
CUDA = True
LR = 1e-4
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
GAMMA = 0.1
VISDOM = False
SAVE_FOLDER = "/media/arg_ws3/5E703E3A703E18EB/research/mm_fcn/ssd/" + DATASET_NAME + "/"
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)

In [3]:
if torch.cuda.is_available():
    if not CUDA:
        print("WTF are u wasting your CUDA device?")
    else:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

## Define function

In [4]:
def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")

# Initial model weights & bias
def xavier(param):
    init.xavier_uniform(param)
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()

# Adjust learning rate during training
def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step 
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    #lr = LR * (gamma ** (step))
    lr = LR * (gamma)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        print("Change learning rate to: ", lr)

## Create dataset

In [5]:
dataset = DATA_DETECTION(root=DATASET_ROOT, image_sets=['train'],transform=SSDAugmentation(cfg['min_dim'], MEANS))

classes = dataset.target_transform.class_to_ind
print("Class to index: \n", classes)
classes = sorted(classes.items(), key=lambda kv: kv[1])
label = []
for i in classes:
    label.append(i[0])
label.append('None')
print(label)
true_label = ['person', 'palm']

Class to index: 
 {'kotex': 9, 'libava': 10, 'swissmiss': 17, 'andes': 1, 'vanish': 18, 'folgers': 4, 'raisins': 15, 'heineken': 5, 'stax': 16, 'hunts': 6, 'crayola': 3, 'viva': 19, 'mm': 13, 'milo': 12, 'macadamia': 11, 'kellogg': 7, 'cocacola': 2, '3m': 0, 'kleenex': 8, 'pocky': 14}
['3m', 'andes', 'cocacola', 'crayola', 'folgers', 'heineken', 'hunts', 'kellogg', 'kleenex', 'kotex', 'libava', 'macadamia', 'milo', 'mm', 'pocky', 'raisins', 'stax', 'swissmiss', 'vanish', 'viva', 'None']


## Create model

In [6]:
ssd_pretrained = build_ssd('train', cfg['min_dim'], 3)
if CUDA:
    net = torch.nn.DataParallel(ssd_pretrained)
    cudnn.benchmark = True
    
if PRETRAINED_MODEL is not None: # Use SSD pretrained model
    print('Resuming training, loading {}...'.format(PRETRAINED_MODEL))
    ssd_pretrained.load_weights(PRETRAINED_MODEL)

/home/arg_ws3/ssd.pytorch/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
/home/arg_ws3/ssd.pytorch/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


In [7]:
# Delcare SSD Network
ssd_net = build_ssd('train', cfg['min_dim'], cfg['num_classes'])
print(cfg['num_classes'])
net = ssd_net
if CUDA:
    net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True
SAME_CLASS = False
if PRETRAINED_MODEL is not None: # Use SSD pretrained model
    if SAME_CLASS:
        print('Resuming training, loading {}...'.format(PRETRAINED_MODEL))
        ssd_net.load_weights(PRETRAINED_MODEL)
    else:
        print('Load pretrained model with different classes')
        ssd_pretrained = build_ssd('train', cfg['min_dim'], 2)
        ssd_pretrained.load_weights(PRETRAINED_MODEL)
        ssd_net.vgg = ssd_pretrained.vgg
        ssd_net.extras = ssd_pretrained.extras
        ssd_net.loc = ssd_pretrained.loc
        ssd_net.conf.apply(weights_init)
else:
    print('Initializing weights...')
    vgg_weights = torch.load(BASE_NET) # load vgg pretrained model
    ssd_net.vgg.load_state_dict(vgg_weights)
    ssd_net.extras.apply(weights_init) # Initial SSD model weights & bias
    ssd_net.loc.apply(weights_init)
    ssd_net.conf.apply(weights_init)

optimizer = optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM,
                weight_decay=WEIGHT_DECAY)
criterion = MultiBoxLoss(BATCH_SIZE ,cfg['num_classes'], 0.5, True, 0, True, 3, 0.5,
                False, CUDA)

21
Initializing weights...


/home/arg_ws3/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  


In [8]:
#for i in range(len(list(ssd_net.parameters()))-30):
#    list(ssd_net.parameters())[i].data = list(ssd_pretrained.parameters())[i]
#list(ssd_net.conf)[-1] = list(ssd_pretrained.conf)[-1]

## Training

### Initialize

In [9]:
net.train()
# loss counters
loc_loss = 0
conf_loss = 0
epoch = 0
print('Loading the dataset...')
epoch_size = len(dataset) // BATCH_SIZE
print('Training SSD on:', DATASET_NAME)

data_loader = data.DataLoader(dataset, BATCH_SIZE,
                                num_workers=NUM_WORKERS,
                                shuffle=True, collate_fn=detection_collate,
                                pin_memory=True)
batch_iterator = iter(data_loader)

Loading the dataset...
Training SSD on: unity


### Start training

In [10]:
iteration = 0
for epoch in range(EPOCH):
    for i, batch in enumerate(data_loader):
        iteration += 1
        images, targets = batch
        if CUDA:
            images = Variable(images.cuda())
            targets = [Variable(ann.cuda(), volatile=True) for ann in targets]
        else:
            images = Variable(images)
            targets = [Variable(ann, volatile=True) for ann in targets]

        # Forward
        t0 = time.time()
        out = net(images)
        # backprop
        optimizer.zero_grad()
        loss_l, loss_c = criterion(out, targets)
        loss = loss_l + loss_c
        loss.backward()
        optimizer.step()
        t1 = time.time()
        loc_loss += loss_l.item()
        conf_loss += loss_c.item()

        if iteration % 8 == 0:
                print('timer: %.4f sec.' % (t1 - t0))
                print('Epoch: ' + str(epoch) + ' || iter ' + repr(PRETRAINED_EPOCH + iteration) + ' || Loss: %.4f ||' % (loss.item()), end='')

    if epoch != 0 and epoch % SAVE_MODEL_EPOCH == 0:
        print('Saving state, Epoch:', epoch)
        torch.save(ssd_net.state_dict(), SAVE_FOLDER + DATASET_NAME + "_" +
                    repr(PRETRAINED_EPOCH + epoch) + '.pth')
    if epoch in adjust_lr_epoch:
        adjust_learning_rate(optimizer, GAMMA, epoch)
    # Save final model
torch.save(ssd_net.state_dict(),
        SAVE_FOLDER + DATASET_NAME + '.pth')

/home/arg_ws3/.local/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/home/arg_ws3/.local/lib/python3.5/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


timer: 0.3176 sec.
Epoch: 0 || iter 8 || Loss: 20.1167 ||timer: 0.3443 sec.
Epoch: 0 || iter 16 || Loss: 15.9807 ||timer: 0.3450 sec.
Epoch: 0 || iter 24 || Loss: 15.4485 ||timer: 0.3454 sec.
Epoch: 0 || iter 32 || Loss: 15.6965 ||timer: 0.3510 sec.
Epoch: 0 || iter 40 || Loss: 15.3060 ||timer: 0.3463 sec.
Epoch: 0 || iter 48 || Loss: 15.1122 ||timer: 0.3404 sec.
Epoch: 0 || iter 56 || Loss: 15.0718 ||timer: 0.3448 sec.
Epoch: 0 || iter 64 || Loss: 14.7454 ||timer: 0.3446 sec.
Epoch: 0 || iter 72 || Loss: 14.7023 ||timer: 0.3451 sec.
Epoch: 0 || iter 80 || Loss: 14.4680 ||timer: 0.3458 sec.
Epoch: 0 || iter 88 || Loss: 14.4668 ||timer: 0.3455 sec.
Epoch: 0 || iter 96 || Loss: 14.2129 ||timer: 0.3460 sec.
Epoch: 0 || iter 104 || Loss: 13.6638 ||timer: 0.3477 sec.
Epoch: 0 || iter 112 || Loss: 13.5415 ||timer: 0.3451 sec.
Epoch: 0 || iter 120 || Loss: 12.5977 ||timer: 0.3446 sec.
Epoch: 0 || iter 128 || Loss: 12.0553 ||timer: 0.3454 sec.
Epoch: 0 || iter 136 || Loss: 11.1842 ||timer: 0.3

Epoch: 0 || iter 1128 || Loss: 3.7300 ||timer: 0.3453 sec.
Epoch: 0 || iter 1136 || Loss: 3.0396 ||timer: 0.3457 sec.
Epoch: 0 || iter 1144 || Loss: 4.2873 ||timer: 0.3473 sec.
Epoch: 0 || iter 1152 || Loss: 3.7101 ||timer: 0.3453 sec.
Epoch: 0 || iter 1160 || Loss: 4.0597 ||timer: 0.3505 sec.
Epoch: 0 || iter 1168 || Loss: 3.9566 ||timer: 0.3459 sec.
Epoch: 0 || iter 1176 || Loss: 4.2746 ||timer: 0.3449 sec.
Epoch: 0 || iter 1184 || Loss: 4.0702 ||timer: 0.3490 sec.
Epoch: 0 || iter 1192 || Loss: 3.5753 ||timer: 0.3456 sec.
Epoch: 0 || iter 1200 || Loss: 3.9895 ||timer: 0.3473 sec.
Epoch: 0 || iter 1208 || Loss: 3.8305 ||timer: 0.3485 sec.
Epoch: 0 || iter 1216 || Loss: 4.2417 ||timer: 0.3450 sec.
Epoch: 0 || iter 1224 || Loss: 3.5624 ||timer: 0.3456 sec.
Epoch: 0 || iter 1232 || Loss: 4.1308 ||timer: 0.3449 sec.
Epoch: 0 || iter 1240 || Loss: 3.7976 ||timer: 0.3457 sec.
Epoch: 0 || iter 1248 || Loss: 3.8136 ||timer: 0.3446 sec.
Epoch: 0 || iter 1256 || Loss: 3.3191 ||timer: 0.3457 se

Epoch: 1 || iter 2240 || Loss: 3.1664 ||timer: 0.3509 sec.
Epoch: 1 || iter 2248 || Loss: 2.7826 ||timer: 0.3468 sec.
Epoch: 1 || iter 2256 || Loss: 2.7527 ||timer: 0.3485 sec.
Epoch: 1 || iter 2264 || Loss: 2.5622 ||timer: 0.3465 sec.
Epoch: 1 || iter 2272 || Loss: 2.6732 ||timer: 0.3459 sec.
Epoch: 1 || iter 2280 || Loss: 2.9931 ||timer: 0.3449 sec.
Epoch: 1 || iter 2288 || Loss: 3.3169 ||timer: 0.3480 sec.
Epoch: 1 || iter 2296 || Loss: 2.6635 ||timer: 0.3471 sec.
Epoch: 1 || iter 2304 || Loss: 3.5064 ||timer: 0.3477 sec.
Epoch: 1 || iter 2312 || Loss: 2.5918 ||timer: 0.3463 sec.
Epoch: 1 || iter 2320 || Loss: 2.6257 ||timer: 0.3446 sec.
Epoch: 1 || iter 2328 || Loss: 2.9889 ||timer: 0.3512 sec.
Epoch: 1 || iter 2336 || Loss: 2.3982 ||timer: 0.3480 sec.
Epoch: 1 || iter 2344 || Loss: 2.7227 ||timer: 0.3463 sec.
Epoch: 1 || iter 2352 || Loss: 2.9142 ||timer: 0.3467 sec.
Epoch: 1 || iter 2360 || Loss: 2.6522 ||timer: 0.3458 sec.
Epoch: 1 || iter 2368 || Loss: 2.4494 ||timer: 0.3468 se

Epoch: 1 || iter 3352 || Loss: 2.6648 ||timer: 0.3455 sec.
Epoch: 1 || iter 3360 || Loss: 2.0127 ||timer: 0.3486 sec.
Epoch: 1 || iter 3368 || Loss: 2.2346 ||timer: 0.3454 sec.
Epoch: 1 || iter 3376 || Loss: 1.9704 ||timer: 0.3461 sec.
Epoch: 1 || iter 3384 || Loss: 1.9005 ||timer: 0.3444 sec.
Epoch: 1 || iter 3392 || Loss: 1.7936 ||timer: 0.3487 sec.
Epoch: 1 || iter 3400 || Loss: 1.9865 ||timer: 0.3503 sec.
Epoch: 1 || iter 3408 || Loss: 2.6244 ||timer: 0.3463 sec.
Epoch: 1 || iter 3416 || Loss: 2.4260 ||timer: 0.3447 sec.
Epoch: 1 || iter 3424 || Loss: 2.2410 ||timer: 0.3461 sec.
Epoch: 1 || iter 3432 || Loss: 2.4929 ||timer: 0.3461 sec.
Epoch: 1 || iter 3440 || Loss: 2.0706 ||timer: 0.3461 sec.
Epoch: 1 || iter 3448 || Loss: 2.1519 ||timer: 0.3490 sec.
Epoch: 1 || iter 3456 || Loss: 1.9646 ||timer: 0.3452 sec.
Epoch: 1 || iter 3464 || Loss: 2.5403 ||timer: 0.3467 sec.
Epoch: 1 || iter 3472 || Loss: 2.1022 ||timer: 0.3458 sec.
Epoch: 1 || iter 3480 || Loss: 2.2632 ||timer: 0.3448 se

...

Epoch: 26 || iter 51928 || Loss: 0.5115 ||timer: 0.3493 sec.
Epoch: 26 || iter 51936 || Loss: 0.5157 ||timer: 0.3462 sec.
Epoch: 26 || iter 51944 || Loss: 0.6692 ||timer: 0.3473 sec.
Epoch: 26 || iter 51952 || Loss: 0.5490 ||timer: 0.3459 sec.
Epoch: 26 || iter 51960 || Loss: 0.5295 ||timer: 0.3500 sec.
Epoch: 26 || iter 51968 || Loss: 0.5716 ||timer: 0.3464 sec.
Epoch: 26 || iter 51976 || Loss: 0.4957 ||timer: 0.3478 sec.
Epoch: 26 || iter 51984 || Loss: 0.6831 ||timer: 0.3469 sec.
Epoch: 26 || iter 51992 || Loss: 0.6651 ||timer: 0.3456 sec.
Epoch: 26 || iter 52000 || Loss: 0.5071 ||timer: 0.3502 sec.
Epoch: 26 || iter 52008 || Loss: 0.5358 ||timer: 0.3465 sec.
Epoch: 26 || iter 52016 || Loss: 0.5314 ||timer: 0.3487 sec.
Epoch: 26 || iter 52024 || Loss: 0.6620 ||timer: 0.3453 sec.
Epoch: 26 || iter 52032 || Loss: 0.5350 ||timer: 0.3483 sec.
Epoch: 26 || iter 52040 || Loss: 0.5586 ||timer: 0.3456 sec.
Epoch: 26 || iter 52048 || Loss: 0.4421 ||timer: 0.3464 sec.
Epoch: 26 || iter 52056 

Epoch: 27 || iter 53000 || Loss: 0.5956 ||timer: 0.3486 sec.
Epoch: 27 || iter 53008 || Loss: 0.6567 ||timer: 0.3504 sec.
Epoch: 27 || iter 53016 || Loss: 0.6652 ||timer: 0.3492 sec.
Epoch: 27 || iter 53024 || Loss: 0.4768 ||timer: 0.3494 sec.
Epoch: 27 || iter 53032 || Loss: 0.5197 ||timer: 0.3480 sec.
Epoch: 27 || iter 53040 || Loss: 0.5679 ||timer: 0.3485 sec.
Epoch: 27 || iter 53048 || Loss: 0.4976 ||timer: 0.3497 sec.
Epoch: 27 || iter 53056 || Loss: 0.4095 ||timer: 0.3468 sec.
Epoch: 27 || iter 53064 || Loss: 0.5616 ||timer: 0.3474 sec.
Epoch: 27 || iter 53072 || Loss: 0.4898 ||timer: 0.3447 sec.
Epoch: 27 || iter 53080 || Loss: 0.6729 ||timer: 0.3468 sec.
Epoch: 27 || iter 53088 || Loss: 0.4679 ||timer: 0.3490 sec.
Epoch: 27 || iter 53096 || Loss: 0.5109 ||timer: 0.3507 sec.
Epoch: 27 || iter 53104 || Loss: 0.5594 ||timer: 0.3469 sec.
Epoch: 27 || iter 53112 || Loss: 0.8783 ||timer: 0.3500 sec.
Epoch: 27 || iter 53120 || Loss: 0.5948 ||timer: 0.3484 sec.
Epoch: 27 || iter 53128 

Epoch: 28 || iter 54072 || Loss: 0.5255 ||timer: 0.3510 sec.
Epoch: 28 || iter 54080 || Loss: 0.5362 ||timer: 0.3485 sec.
Epoch: 28 || iter 54088 || Loss: 0.4812 ||timer: 0.3471 sec.
Epoch: 28 || iter 54096 || Loss: 0.5950 ||timer: 0.3513 sec.
Epoch: 28 || iter 54104 || Loss: 0.4037 ||timer: 0.3495 sec.
Epoch: 28 || iter 54112 || Loss: 0.4960 ||timer: 0.3449 sec.
Epoch: 28 || iter 54120 || Loss: 0.5790 ||timer: 0.3457 sec.
Epoch: 28 || iter 54128 || Loss: 0.5241 ||timer: 0.3518 sec.
Epoch: 28 || iter 54136 || Loss: 0.5960 ||timer: 0.3509 sec.
Epoch: 28 || iter 54144 || Loss: 0.5065 ||timer: 0.3500 sec.
Epoch: 28 || iter 54152 || Loss: 0.6171 ||timer: 0.3454 sec.
Epoch: 28 || iter 54160 || Loss: 0.5461 ||timer: 0.3455 sec.
Epoch: 28 || iter 54168 || Loss: 0.6139 ||timer: 0.3519 sec.
Epoch: 28 || iter 54176 || Loss: 0.5117 ||timer: 0.3461 sec.
Epoch: 28 || iter 54184 || Loss: 0.6487 ||timer: 0.3454 sec.
Epoch: 28 || iter 54192 || Loss: 0.4844 ||timer: 0.3451 sec.
Epoch: 28 || iter 54200 

Exception in thread Thread-33:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/home/arg_ws3/.local/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 158, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 113, in get
    return ForkingPickler.loads(res)
  File "/home/arg_ws3/.local/lib/python3.5/site-packages/torch/multiprocessing/reductions.py", line 256, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.5/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.5/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
  File "/

KeyboardInterrupt: 

In [ ]:
'''step_index = 0
for iteration in range(START_ITER, cfg['max_iter']):
    if iteration in cfg['lr_steps']:
        step_index += 1
        adjust_learning_rate(optimizer, GAMMA, step_index)
    
    # make sure data iter not out of range
    try:
        images, targets = next(batch_iterator)
        #print(targets[0][0][4].item(), label[int(targets[0][0][4].item())])
    except StopIteration:
        batch_iterator = iter(data_loader)
        images, targets = next(batch_iterator)
    if CUDA:
        images = Variable(images.cuda())
        targets = [Variable(ann.cuda(), volatile=True) for ann in targets]
    else:
        images = Variable(images)
        targets = [Variable(ann, volatile=True) for ann in targets]
    
    # Forward
    t0 = time.time()
    out = net(images)
    # backprop
    optimizer.zero_grad()
    loss_l, loss_c = criterion(out, targets)
    loss = loss_l + loss_c
    loss.backward()
    optimizer.step()
    t1 = time.time()
    loc_loss += loss_l.item()
    conf_loss += loss_c.item()
    
    if iteration % 10 == 0:
            print('timer: %.4f sec.' % (t1 - t0))
            print('iter ' + repr(PRETRAINED_ITER + iteration) + ' || Loss: %.4f ||' % (loss.item()), end='')
    
    if iteration != 0 and iteration % SAVE_MODEL_ITER == 0:
            print('Saving state, iter:', iteration)
            torch.save(ssd_net.state_dict(), SAVE_FOLDER + DATASET_NAME + "_" +
                       repr(PRETRAINED_ITER + iteration) + '.pth')
# Save final model
torch.save(ssd_net.state_dict(),
            SAVE_FOLDER + DATASET_NAME + '.pth')'''

In [ ]:
torch.cuda.empty_cache()